Bag of Binary Words

In [36]:
import glob
import random
random.seed(123123)

In [37]:
import cv2
import numpy as np


In [38]:
import dbow


In [39]:
# Load Images
images_path = glob.glob('./images/*.png')
images = []
for image_path in images_path:
    images.append(cv2.imread(image_path))


In [40]:
# Create Vocabulary
n_clusters = 10
depth = 2
vocabulary = dbow.Vocabulary(images, n_clusters, depth)


In [41]:
orb = cv2.ORB_create()


In [42]:
# Convert images to Bag of Binary Words and calculate scores between them
bows = []
for image in images:
    kps, descs = orb.detectAndCompute(image, None)
    descs = [dbow.ORB.from_cv_descriptor(desc) for desc in descs]
    bows.append(vocabulary.descs_to_bow(descs))

for i in range(len(bows)):
    for j in range(len(bows)):
        print(f'Similarity between Image {i} and Image {j} = {bows[i].score(bows[j])}')
    print('\n')


Similarity between Image 0 and Image 0 = 1.0
Similarity between Image 0 and Image 1 = 0.7164073555028767
Similarity between Image 0 and Image 2 = 0.5579603609712325
Similarity between Image 0 and Image 3 = 0.49411308656281794


Similarity between Image 1 and Image 0 = 0.7164073555028767
Similarity between Image 1 and Image 1 = 1.0
Similarity between Image 1 and Image 2 = 0.437246127028442
Similarity between Image 1 and Image 3 = 0.3871140542647854


Similarity between Image 2 and Image 0 = 0.5579603609712325
Similarity between Image 2 and Image 1 = 0.437246127028442
Similarity between Image 2 and Image 2 = 1.0
Similarity between Image 2 and Image 3 = 0.5251429384400262


Similarity between Image 3 and Image 0 = 0.49411308656281794
Similarity between Image 3 and Image 1 = 0.3871140542647854
Similarity between Image 3 and Image 2 = 0.5251429384400262
Similarity between Image 3 and Image 3 = 1.0




In [43]:
# Create a database
db = dbow.Database(vocabulary)
for image in images:
    kps, descs = orb.detectAndCompute(image, None)
    descs = [dbow.ORB.from_cv_descriptor(desc) for desc in descs]
    db.add(descs)


In [44]:
# Query the database
for image in images:
    kps, descs = orb.detectAndCompute(image, None)
    descs = [dbow.ORB.from_cv_descriptor(desc) for desc in descs]
    scores = db.query(descs)
    match_bow = db[np.argmax(scores)]
    match_desc = db.descriptors[np.argmax(scores)]


In [45]:
# Saving and Loading the vocabulary
vocabulary.save('vocabulary.pickle')
loaded_vocabulary = vocabulary.load('vocabulary.pickle')
for image in images:
    kps, descs = orb.detectAndCompute(image, None)
    descs = [dbow.ORB.from_cv_descriptor(desc) for desc in descs]
    loaded_vocabulary.descs_to_bow(descs)
    

In [ ]:
# Saving and Loading the database
db.save('database.pickle')
loaded_db = db.load('database.pickle')
for image in images:
    kps, descs = orb.detectAndCompute(image, None)
    descs = [dbow.ORB.from_cv_descriptor(desc) for desc in descs]
    scores = loaded_db.query(descs)
    print(loaded_db[np.argmax(scores)], np.argmax(scores))